In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import tensorflow as tf

This model will use logistic regression to classify a day with frost based on past weather conditions.

In [2]:
# Reads in example csv file to test if working
# Outcome which day will have frost, 1 means there was frost 0 means no frost
File_path2 = "../resources/observation_data.csv"
observation_df = pd.read_csv(File_path2)
observation_df

,station_uid,date,maxt,mint,pcpn,snow,snwd,avgt,freeze_day,above_freezing,...,maxt_7_day,mint_7_day,avgt_7_day,precip_7_day,obs_count_7_day,maxt_30_day,mint_30_day,avgt_30_day,precip_30_day,obs_count_30_day
0,10392,2003-03-05,43.0,28.0,0.00,NaN,NaN,35.5,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10392,2003-03-06,43.0,31.0,0.00,NaN,NaN,37.0,1,1,...,43.0,28.0,35.500000,0.00,1.0,43.0,28.0,35.500000,0.00,1.0
2,10392,2003-03-07,45.0,20.0,0.00,NaN,NaN,32.5,1,1,...,43.0,28.0,36.250000,0.00,2.0,43.0,28.0,36.250000,0.00,2.0
3,10392,2003-03-08,59.0,32.0,0.00,NaN,NaN,45.5,1,1,...,45.0,20.0,35.000000,0.00,3.0,45.0,20.0,35.000000,0.00,3.0
4,10392,2003-03-09,55.0,30.0,0.00,NaN,NaN,42.5,1,1,...,59.0,20.0,37.625000,0.00,4.0,59.0,20.0,37.625000,0.00,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6407,10685,2004-05-12,77.0,63.0,1.65,0.0,NaN,70.0,0,1,...,85.0,40.0,61.428571,0.10,7.0,85.0,21.0,46.500000,3.25,30.0
6408,10685,2004-05-13,92.0,52.0,0.13,NaN,NaN,72.0,0,1,...,85.0,46.0,64.000000,1.75,7.0,85.0,21.0,47.200000,4.90,30.0
6409,10685,2004-05-14,75.0,56.0,0.00,0.0,NaN,65.5,0,1,...,92.0,46.0,66.357143,1.88,7.0,92.0,21.0,47.950000,4.35,30.0
6410,10685,2004-05-15,74.0,57.0,0.00,0.0,NaN,65.5,0,1,...,92.0,52.0,67.285714,1.88,7.0,92.0,21.0,48.883333,4.35,30.0


In [3]:
observation_df = observation_df.drop(columns="date")
observation_df.head(10)

,station_uid,maxt,mint,pcpn,snow,snwd,avgt,freeze_day,above_freezing,obs_year,...,maxt_7_day,mint_7_day,avgt_7_day,precip_7_day,obs_count_7_day,maxt_30_day,mint_30_day,avgt_30_day,precip_30_day,obs_count_30_day
0,10392,43.0,28.0,0.00,NaN,NaN,35.5,1,1,2003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10392,43.0,31.0,0.00,NaN,NaN,37.0,1,1,2003,...,43.0,28.0,35.500000,0.00,1.0,43.0,28.0,35.500000,0.00,1.0
2,10392,45.0,20.0,0.00,NaN,NaN,32.5,1,1,2003,...,43.0,28.0,36.250000,0.00,2.0,43.0,28.0,36.250000,0.00,2.0
3,10392,59.0,32.0,0.00,NaN,NaN,45.5,1,1,2003,...,45.0,20.0,35.000000,0.00,3.0,45.0,20.0,35.000000,0.00,3.0
4,10392,55.0,30.0,0.00,NaN,NaN,42.5,1,1,2003,...,59.0,20.0,37.625000,0.00,4.0,59.0,20.0,37.625000,0.00,4.0
5,10392,62.0,40.0,0.00,NaN,NaN,51.0,0,1,2003,...,59.0,20.0,38.600000,0.00,5.0,59.0,20.0,38.600000,0.00,5.0
6,10392,55.0,42.0,0.83,NaN,NaN,48.5,0,1,2003,...,62.0,20.0,40.666667,0.00,6.0,62.0,20.0,40.666667,0.00,6.0
7,10392,48.0,32.0,0.00,NaN,NaN,40.0,1,1,2003,...,62.0,20.0,41.785714,0.83,7.0,62.0,20.0,41.785714,0.83,7.0
8,10392,45.0,26.0,0.00,NaN,NaN,35.5,1,1,2003,...,62.0,20.0,42.428571,0.83,7.0,62.0,20.0,41.562500,0.83,8.0
9,10392,59.0,27.0,0.00,NaN,NaN,43.0,1,1,2003,...,62.0,20.0,42.214286,0.83,7.0,62.0,20.0,40.888889,0.83,9.0


In [4]:
observation_df.isnull().sum(axis = 0)

station_uid            0
maxt                   6
mint                   0
pcpn                 328
snow                2255
snwd                3183
avgt                   6
freeze_day             0
above_freezing         0
obs_year               0
obs_month              0
obs_day                0
obs_dayofyear          0
maxt_7_day            11
mint_7_day            11
avgt_7_day            11
precip_7_day          11
obs_count_7_day       11
maxt_30_day            9
mint_30_day            9
avgt_30_day            9
precip_30_day          9
obs_count_30_day       9
dtype: int64

In [5]:
# To not lose too many rows with NaN snow and precipitation amount with NaN values will be replaced with 0
observation_df["snow"] = observation_df["snow"].fillna(0)
observation_df["snwd"] = observation_df["snwd"].fillna(0)
observation_df["pcpn"] = observation_df["pcpn"].fillna(0)
observation_df.head(10)

,station_uid,maxt,mint,pcpn,snow,snwd,avgt,freeze_day,above_freezing,obs_year,...,maxt_7_day,mint_7_day,avgt_7_day,precip_7_day,obs_count_7_day,maxt_30_day,mint_30_day,avgt_30_day,precip_30_day,obs_count_30_day
0,10392,43.0,28.0,0.00,0.0,0.0,35.5,1,1,2003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10392,43.0,31.0,0.00,0.0,0.0,37.0,1,1,2003,...,43.0,28.0,35.500000,0.00,1.0,43.0,28.0,35.500000,0.00,1.0
2,10392,45.0,20.0,0.00,0.0,0.0,32.5,1,1,2003,...,43.0,28.0,36.250000,0.00,2.0,43.0,28.0,36.250000,0.00,2.0
3,10392,59.0,32.0,0.00,0.0,0.0,45.5,1,1,2003,...,45.0,20.0,35.000000,0.00,3.0,45.0,20.0,35.000000,0.00,3.0
4,10392,55.0,30.0,0.00,0.0,0.0,42.5,1,1,2003,...,59.0,20.0,37.625000,0.00,4.0,59.0,20.0,37.625000,0.00,4.0
5,10392,62.0,40.0,0.00,0.0,0.0,51.0,0,1,2003,...,59.0,20.0,38.600000,0.00,5.0,59.0,20.0,38.600000,0.00,5.0
6,10392,55.0,42.0,0.83,0.0,0.0,48.5,0,1,2003,...,62.0,20.0,40.666667,0.00,6.0,62.0,20.0,40.666667,0.00,6.0
7,10392,48.0,32.0,0.00,0.0,0.0,40.0,1,1,2003,...,62.0,20.0,41.785714,0.83,7.0,62.0,20.0,41.785714,0.83,7.0
8,10392,45.0,26.0,0.00,0.0,0.0,35.5,1,1,2003,...,62.0,20.0,42.428571,0.83,7.0,62.0,20.0,41.562500,0.83,8.0
9,10392,59.0,27.0,0.00,0.0,0.0,43.0,1,1,2003,...,62.0,20.0,42.214286,0.83,7.0,62.0,20.0,40.888889,0.83,9.0


In [6]:
observation_df = observation_df.dropna()

In [7]:
observation_df = observation_df.reset_index(drop=True)

In [8]:
observation_df

,station_uid,maxt,mint,pcpn,snow,snwd,avgt,freeze_day,above_freezing,obs_year,...,maxt_7_day,mint_7_day,avgt_7_day,precip_7_day,obs_count_7_day,maxt_30_day,mint_30_day,avgt_30_day,precip_30_day,obs_count_30_day
0,10392,43.0,31.0,0.00,0.0,0.0,37.0,1,1,2003,...,43.0,28.0,35.500000,0.00,1.0,43.0,28.0,35.500000,0.00,1.0
1,10392,45.0,20.0,0.00,0.0,0.0,32.5,1,1,2003,...,43.0,28.0,36.250000,0.00,2.0,43.0,28.0,36.250000,0.00,2.0
2,10392,59.0,32.0,0.00,0.0,0.0,45.5,1,1,2003,...,45.0,20.0,35.000000,0.00,3.0,45.0,20.0,35.000000,0.00,3.0
3,10392,55.0,30.0,0.00,0.0,0.0,42.5,1,1,2003,...,59.0,20.0,37.625000,0.00,4.0,59.0,20.0,37.625000,0.00,4.0
4,10392,62.0,40.0,0.00,0.0,0.0,51.0,0,1,2003,...,59.0,20.0,38.600000,0.00,5.0,59.0,20.0,38.600000,0.00,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6390,10685,77.0,63.0,1.65,0.0,0.0,70.0,0,1,2004,...,85.0,40.0,61.428571,0.10,7.0,85.0,21.0,46.500000,3.25,30.0
6391,10685,92.0,52.0,0.13,0.0,0.0,72.0,0,1,2004,...,85.0,46.0,64.000000,1.75,7.0,85.0,21.0,47.200000,4.90,30.0
6392,10685,75.0,56.0,0.00,0.0,0.0,65.5,0,1,2004,...,92.0,46.0,66.357143,1.88,7.0,92.0,21.0,47.950000,4.35,30.0
6393,10685,74.0,57.0,0.00,0.0,0.0,65.5,0,1,2004,...,92.0,52.0,67.285714,1.88,7.0,92.0,21.0,48.883333,4.35,30.0


In [9]:
# Separate the Features (X) from the Target (y)
y =observation_df["freeze_day"]
x =observation_df.drop(columns="freeze_day")

In [10]:
# Split our data into training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(4796, 22)

In [11]:
# Create a Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)

In [12]:
# Fit (train) or model using the training data
classifier.fit(X_train, y_train)

c:\Users\bmwil\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(max_iter=200, random_state=1)

In [13]:
# Make predictions, 1 means there was frost 0 means no frost
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(20)

,Prediction,Actual
0,0,0
1,1,1
2,0,0
3,0,0
4,0,0
5,1,1
6,1,1
7,1,1
8,1,1
9,1,1


In [14]:
# Accuracy metric
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

1.0
